In [ ]:
import os
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from PIL import Image
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
# param definitions and options

z_dim = 50
b_size = 200
max_iter = 1000
step_size = 1e-3
filter_size = 9
max_rotation = .1
fst_lyr_num_fltrs = 64
scnd_lyr_num_fltrs = 64

sample_batch = 1

loss_to_use = 'l_1'; # options: 'l2_squared', 'l_1', 'cosine'
#use_cosine = False
use_leaky_relu = False
use_tensorboard = True
use_rotation = False
use_warmstart = True
save_model = True

modelload_directory = './models/generator/test_zdim_50_8_more_var/'
modelsave_directory = './models/generator/test_zdim_50_9_more_var/'
tensorboard_dir = './logs/tensorboard/test_zdim_50_9_more_var'
sample_dir = 'logs/test_zdim_50_9_more_var'

# defining the sample directory if it does not exist
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)


In [ ]:
# helper functions

def data2img(data):
    shape = [28, 28, 1]
    return np.reshape(data, [data.shape[0]] + shape)

def grid_transform(x, size):
    #a, t_b = split(x.shape[0])
    #b = int(t_b)
    a = 10
    b = int(x.shape[0]/a)
    h, w, c = size[0], size[1], size[2]
    x = np.reshape(x, [a, b, h, w, c])
    x = np.transpose(x, [0, 2, 1, 3, 4])
    x = np.reshape(x, [a * h, b * w, c])
    if x.shape[2] == 1:
        x = np.squeeze(x, axis=2)
    return x

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def weight_placeholder(shape, name=None):
    return tf.placeholder(tf.float32, shape, name = name)

def leaky_relu(x, alpha=0.2):
    return tf.maximum(x * alpha, x)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def bias_placeholder(shape, name = None):
    return tf.placeholder(tf.float32, shape, name = name)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def return_normal(shape):
    return np.float32(np.random.normal(loc=0.0, scale=0.1, size=shape))
    

def return_const(shape, c = .1):
    return np.float32(np.full(shape = shape, fill_value = c))

def return_unifrom(shape):
    return np.float32(np.random.uniform(-1.0, 1.0, shape))

def grad_placeholder(g):
    g_placeholder = []
    ctr = 0
    for t in g:
        #if use_warmstart:
        #    graph = tf.get_default_graph()
        #    g_placeholder.append(graph.get_tensor_by_name('grad_p'+str(ctr)+':0'))
        #else:
        g_placeholder.append(tf.placeholder(tf.float32, shape = t.get_shape().as_list(), name = 'grad_p'+str(ctr)))
        ctr = ctr+1
    return g_placeholder

def feed_for_grad(g):
    ctr = 0
    feed={}
    for t in g:
        feed['grad_p'+str(ctr)+':0'] = t
        ctr = ctr + 1
    return feed

def dist_d(a, b):
    if loss_to_use == 'cosine':
    #if use_cosine:
        norms0 = tf.matmul(tf.sqrt(tf.reduce_sum(a**2,1, keep_dims=True)), tf.sqrt(tf.reduce_sum(b**2,1, keep_dims=True)), transpose_b=True) + 1e-5
        return (1. - tf.matmul(a, b, transpose_b=True)/(norms0))*500.0
    elif loss_to_use == 'l_1':
        a = tf.expand_dims(a, 0)
        b = tf.expand_dims(b, 1)
        return tf.reduce_mean(tf.abs(a-b), 2)*500.
    else:
        a = tf.expand_dims(a, 0)
        b = tf.expand_dims(b, 1)
        return tf.reduce_mean((a-b)**2, 2)*500.

#def initialize_uninitialized_vars(sess):
#    from itertools import compress
#    global_vars = tf.global_variables()
#    is_not_initialized = sess.run([~(tf.is_variable_initialized(var)) \
#                                   for var in global_vars])
#    not_initialized_vars = list(compress(global_vars, is_not_initialized))
#
#    if len(not_initialized_vars):
#        sess.run(tf.variables_initializer(not_initialized_vars))



In [ ]:
# make the session

tf.reset_default_graph()
sess = tf.InteractiveSession()

In [ ]:
# define the generator

if use_warmstart:
    saver = tf.train.import_meta_graph(modelload_directory+'model.meta')
    saver.restore(sess,tf.train.latest_checkpoint(modelload_directory))
    gen_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "Generator") 
    
    gen_params_to_restore = sess.run(gen_params)
    tf.reset_default_graph()
    sess.close()
    print(gen_params)
    #print(len(gen_params_to_restore))
    #for t in gen_params_to_restore:
    #    print(t.shape)
    
    warm_init_dict = {}
    ctr = 0
    for var in gen_params:
        warm_init_dict[var.name] = gen_params_to_restore[ctr]
        ctr = ctr + 1
    #print(warm_init_dict)
    sess = tf.InteractiveSession()
    def return_warm_init(var_name):
        value = warm_init_dict[var_name]
        return tf.constant_initializer(value)

def Generator(z, reuse = False):
    with tf.variable_scope("Generator", reuse=reuse):
        if use_warmstart:
            bs = tf.shape(z)[0]
            fc1 = tf.layers.dense(z, 1024, 
                                 kernel_initializer = return_warm_init('Generator/dense/kernel:0'),
                                 bias_initializer = return_warm_init('Generator/dense/bias:0'))
            fc1 = tf.nn.relu(fc1)
            fc2 = tf.layers.dense(fc1, 7 * 7 * 128, 
                                  kernel_initializer = return_warm_init('Generator/dense_1/kernel:0'),
                                 bias_initializer = return_warm_init('Generator/dense_1/bias:0'))
            fc2 = tf.reshape(fc2, tf.stack([bs, 7, 7, 128]))
            fc2 = tf.nn.relu(fc2)
            conv1 = tf.contrib.layers.conv2d_transpose(fc2, 64, [4, 4], [2, 2],
                                                      weights_initializer = return_warm_init('Generator/Conv2d_transpose/weights:0'),
                                                      biases_initializer = return_warm_init('Generator/Conv2d_transpose/biases:0'))
            conv1 = tf.nn.relu(conv1)
            conv2 = tf.contrib.layers.conv2d_transpose(conv1, 1, [4, 4], [2, 2], activation_fn=tf.sigmoid,
                                                      weights_initializer = return_warm_init('Generator/Conv2d_transpose_1/weights:0'),
                                                      biases_initializer = return_warm_init('Generator/Conv2d_transpose_1/biases:0'))
            conv2 = tf.reshape(conv2, tf.stack([bs, 784]))
            
        else:
            bs = tf.shape(z)[0]
            fc1 = tf.layers.dense(z, 1024)
            fc1 = tf.nn.relu(fc1)
            fc2 = tf.layers.dense(fc1, 7 * 7 * 128)
            fc2 = tf.reshape(fc2, tf.stack([bs, 7, 7, 128]))
            fc2 = tf.nn.relu(fc2)
            conv1 = tf.contrib.layers.conv2d_transpose(fc2, 64, [4, 4], [2, 2])
            conv1 = tf.nn.relu(conv1)
            conv2 = tf.contrib.layers.conv2d_transpose(conv1, 1, [4, 4], [2, 2], activation_fn=tf.sigmoid)
            conv2 = tf.reshape(conv2, tf.stack([bs, 784]))
            
    return conv2
    

In [ ]:
# defining the discriminator

W_conv1 = weight_placeholder([filter_size, filter_size, 1, fst_lyr_num_fltrs], name = 'W_conv1' )
b_conv1 = bias_placeholder([fst_lyr_num_fltrs], name = 'b_conv1')

W_conv2 = weight_placeholder([filter_size, filter_size, fst_lyr_num_fltrs, scnd_lyr_num_fltrs], name = 'W_conv2')
b_conv2 = bias_placeholder([scnd_lyr_num_fltrs], name = 'b_conv2')

if use_rotation:
    angle = tf.placeholder(tf.float32, [1], name = 'angle')


def Create_feed_dict():
    feed = {}
    if use_rotation:
        feed[angle] = max_rotation*return_unifrom([1]);
    feed[W_conv1] = return_normal([filter_size, filter_size, 1, fst_lyr_num_fltrs])
    feed[b_conv1] = return_const([fst_lyr_num_fltrs], c = 0.0)
    feed[W_conv2] = return_normal([filter_size, filter_size, fst_lyr_num_fltrs, scnd_lyr_num_fltrs])
    feed[b_conv2] = return_const([scnd_lyr_num_fltrs], c = 0.0 )
    #feed['z1:0'] = return_unifrom([b_size , z_dim])
    #feed['z2:0'] = return_unifrom([b_size , z_dim])
    return feed

def Discriminator(x, angle = 0.0):
    x_image = tf.reshape(x, [-1,28,28,1])
    
    if use_rotation:
        x_image = tf.contrib.image.rotate(x_image, angle)

    ##  #first layer
    if use_leaky_relu:
        h_conv1 = leaky_relu(conv2d(x_image, W_conv1) + b_conv1)
    else:
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    
    h_pool1 = max_pool_2x2(h_conv1)


    ##  #second layer
    
    if use_leaky_relu:
        h_conv2 = leaky_relu(conv2d(h_pool1, W_conv2) + b_conv2)
    else:
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*scnd_lyr_num_fltrs])
    #h_pool2_flat = tf.reshape(h_pool2, [-1, 14*14*64])
    return h_pool2_flat

In [ ]:
# main
    
x1 = tf.placeholder(tf.float32, [None, 784], name = 'x1')
x2 = tf.placeholder(tf.float32, [None, 784], name = 'x2')

z1 = tf.placeholder(tf.float32, [None, z_dim], name = 'z1')
z2 = tf.placeholder(tf.float32, [None, z_dim], name = 'z2')

gz1 = Generator(z1)
gz1 = tf.identity(gz1, name = 'gz1_n')
gz2 = Generator(z2, reuse = True)
gz2 = tf.identity(gz2, name = 'gz2_n')

    
if use_rotation:
    
    #angle = tf.placeholder(tf.float32, [1], name = 'angle')
    
    dx1 = Discriminator(x1, angle = angle)
    dx2 = Discriminator(x2, angle = angle)


    dz1 = Discriminator(gz1, angle = angle)
    dz2 = Discriminator(gz2, angle = angle)
    
else:    
    
    dx1 = Discriminator(x1)
    dx2 = Discriminator(x2)


    dz1 = Discriminator(gz1)
    dz2 = Discriminator(gz2)

g_temp1 = dist_d(dx1, dx2)
g_temp2 = dist_d(dz1, dz2)
g_temp3 = dist_d(dx1, dz1)
g_temp4 = dist_d(dx2, dz2)

g_loss = tf.reduce_mean(g_temp4+ g_temp3- g_temp2 - g_temp1)



if use_tensorboard:
    #tf.summary.histogram("g_loss", g_loss)
    tf.summary.scalar('cost', g_loss)

gen_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "Generator") 
print(gen_params)

gen_grads = tf.gradients(g_loss, gen_params)
print(gen_grads)

grads_placeholder = grad_placeholder(gen_grads)#tf.placeholder(tf.float32, tf.shape(gen_grads), name = 'grads')

print(grads_placeholder)
print(gen_grads)

#gen_train_op = tf.train.AdamOptimizer(learning_rate=step_size, beta1=0.5, beta2=0.9).minimize(g_loss, var_list=gen_params)

#if use_warmstart:
#    graph = tf.get_default_graph()
#    gen_train_op = graph.get_operation_by_name("Adam1") 
    
gen_train_op = tf.train.AdamOptimizer(learning_rate=step_size,
                                      beta1=0.5,
                                      beta2=0.9, name = 'Adam2').apply_gradients(zip(grads_placeholder, gen_params))
    

In [ ]:
# a helper function that saves generated samples

def Generate_samples_and_save(fixed_noise, iter = 0):
    fake_samples = sess.run(gz1, feed_dict={z1:fixed_noise})
    fake_samples = data2img(fake_samples)
    fake_samples = grid_transform(fake_samples, [28, 28, 1])
    fake_samples = np.squeeze(fake_samples)
    #fake_samples = (255.99*fake_samples).astype('uint8')
    plt.imsave(sample_dir+'/samples_'+str(iter)+'.png', fake_samples)
    img = Image.open(sample_dir+'/samples_'+str(iter)+'.png').convert('LA')
    img.save(sample_dir+'/samples_'+str(iter)+'.png')
    return fake_samples
    

In [ ]:
# learning


fixed_noise = np.random.uniform(-1.0, 1.0, [120, z_dim])


if use_tensorboard:
    train_writer = tf.summary.FileWriter( tensorboard_dir, sess.graph)

#if use_warmstart:
#    initialize_uninitialized_vars(sess)
#else:
sess.run(tf.global_variables_initializer())

cost = []
norm_grad = []
for iter in range(max_iter):
    for j in range(sample_batch):
        feed = Create_feed_dict()
        data1 = mnist.train.next_batch(b_size)[0]
        data2 = mnist.train.next_batch(b_size)[0]
        code1 = return_unifrom([b_size , z_dim])
        code2 = return_unifrom([b_size , z_dim])
        feed[x1] = data1
        feed[x2] = data2
        feed[z1] = code1
        feed[z2] = code2
        
        if j==0:
            cost_ , g_ = sess.run([g_loss, gen_grads], feed_dict = feed)
            
        else:
            cost_j, g_j = sess.run([g_loss, gen_grads], feed_dict = feed)
            cost_ = cost_ + cost_j
            for i in range(len(g_)):
                g_[i] = g_[i] + g_j[i]
    
    cost_ = (1.0/sample_batch)*cost_
    norm_grad_i = 0
    for t in g_:
        t = (1.0/sample_batch)*t
        norm_grad_i = norm_grad_i + np.sum(np.power(t,2))
    norm_grad_i = np.sqrt(norm_grad_i)
        
    cost.append(cost_)
    norm_grad.append(norm_grad_i)

    _ = sess.run(gen_train_op, feed_dict=feed_for_grad(g_))    
    
    
    if use_tensorboard:
        #summary_writer = tf.train.SummaryWriter(FLAGS.logdir)
        summary = tf.Summary()
        summary.value.add(tag='gen cost', simple_value=cost_)
        summary.value.add(tag='grad norm', simple_value=norm_grad_i)
        train_writer.add_summary(summary, iter)
        #summary_writer.flush()

    #if use_tensorboard:
    #    merge = tf.summary.merge_all()
    #    summary, _ = sess.run([merge, gen_train_op], feed_dict = feed)
    #    train_writer.add_summary(summary, iter)
    #else:
    #    cost_, _ = sess.run([g_loss, gen_train_op], feed_dict = feed)
    #    cost.append(cost_)
    
    if iter%10==0:
        print('iter: ' + str(iter)+ ', cost: '+ str(cost[-1]) + ', grad norm: ' + str(norm_grad[-1]) )
    if iter%100==0:
        Generate_samples_and_save(fixed_noise, iter = iter)
    
Generate_samples_and_save(fixed_noise, iter = iter)



In [ ]:
# save the model
if save_model:
    saver_gen = tf.train.Saver(gen_params)
    saver_gen.save(sess, modelsave_directory + 'model')
    #saver = tf.train.Saver()
    #tf.add_to_collection('bottle', h_fc1)
    #saver.save(sess, modelsave_directory + 'model')
    